In [8]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import os
import logging

In [9]:
logging.basicConfig(filename='log.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s', encoding='utf-8')

In [10]:
def modify_block_number(x):
    if x == 1:
        return 1
    elif x in [2, 3, 4]:
        return 2
    elif x == 5:
        return 3
    elif x in [6, 7, 8]:
        return 4
    elif x == 9:
        return 5
    elif x in [10, 11, 12]:
        return 6
    elif x == 13:
        return 7
    elif x in [14, 15, 16]:
        return 8
    else:
        return None  # На случай, если значение не входит в указанные диапазоны

def add_iat_exp(x):
    if x in [1, 3, 5, 7]:
        return 'train'
    elif x in [2, 4, 6, 8]:
        return 'main'
    else:
        return None

def process_trials(df):
    logging.info('Рассчёт RT и обработка button...')
    # Группируем строки по 'Total Trial Number'
    grouped = df.groupby('Total Trial Number')
    
    # Для каждой группы (блока) считаем сумму RT и проверяем наличие wrong button
    for name, group in grouped:
        total_rt = group['RT'].sum()  # Считаем сумму RT для блока
                
        # Заменяем значения RT на сумму RT для всех строк группы
        df.loc[df['Total Trial Number'] == name, 'RT'] = total_rt
        
        # Если в блоке есть хотя бы один wrong button, заменяем все correct button на wrong button
        if (group['Trial Result'] == 'wrong button').any(): # Проверяем наличие wrong button
            df.loc[(df['Total Trial Number'] == name) & (df['Trial Result'] == 'correct button'), 'Trial Result'] = 'wrong button'
    return df

def add_iat_acc(x):
    if x == 'correct button':
        return 1 
    elif x == 'wrong button':
        return 0
    else:
        return None

def add_trial_valence(row):
    if row['Block Number Modified'] in [1, 2, 7, 8]:
        return 'positive'
    elif row['Block Number Modified'] in [3, 4, 5, 6]:
        return 'negative'
    else:
        return None

def determine_iat_cb(row):
    if row['IAT_results'] > 0 and row['Trial Valence'] == 'positive':
        return 'match'
    elif row['IAT_results'] < 0 and row['Trial Valence'] == 'negative':
        return 'match'
    elif row['IAT_results'] < 0 and row['Trial Valence'] == 'positive':
        return 'mismatch'
    elif row['IAT_results'] > 0 and row['Trial Valence'] == 'negative':
        return 'mismatch'
    else:
        return None 

def Vac2Dataset(df, xls, fixation_sheets, saccade_sheets):
    df = df.copy()
    # Создание списка для хранения строк
    rows = []

    logging.info('Read Fixations...')
    fixation_sheets_data = [pd.read_excel(xls, sheet_name=sheet) for sheet in tqdm(fixation_sheets, total=len(fixation_sheets))]
    logging.info('Read Saccades...')
    saccade_sheets_data = [pd.read_excel(xls, sheet_name=sheet) for sheet in tqdm(saccade_sheets, total=len(saccade_sheets))]
    logging.info('Processing fixation and saccade lists...')
    # Для каждого индекса Base_Report будем копировать строки столько раз, сколько нужно для фиксации и саккад
    for idx in tqdm(range(df.shape[0]), total=df.shape[0]):
        # Получаем текущие данные Fixation и Saccade
        fixation_df = fixation_sheets_data[idx]
        saccade_df = saccade_sheets_data[idx]

        # Получаем текущую строку Base_Report
        base_row = df.iloc[[idx]].copy()  # Копируем строку как DataFrame

        # Определяем количество строк в Fixation и Saccade для данного индекса
        fixation_rows = fixation_df if not fixation_df.empty else pd.DataFrame(columns=fixation_df.columns)
        saccade_rows = saccade_df if not saccade_df.empty else pd.DataFrame(columns=saccade_df.columns)

        # Максимум строк для этого индекса
        max_rows_local = max(len(fixation_rows), len(saccade_rows))

        # Если ни Fixation, ни Saccade не имеют строк, просто добавляем строку Base_Report
        if max_rows_local == 0:
            rows.append(base_row)

        for i in range(max_rows_local):
            # Создаем копию строки Base_Report для каждой итерации
            new_row = base_row.copy()

            # Добавляем данные из Fixation, если они есть
            if i < len(fixation_rows):
                for col in fixation_df.columns:
                    new_row[f"f_{col}"] = fixation_rows.iloc[i][col]
            else:
                for col in fixation_df.columns:
                    new_row[f"f_{col}"] = None

            # Добавляем данные из Saccade, если они есть
            if i < len(saccade_rows):
                for col in saccade_df.columns:
                    new_row[f"s_{col}"] = saccade_rows.iloc[i][col]
            else:
                for col in saccade_df.columns:
                    new_row[f"s_{col}"] = None

            # Добавляем строку в список
            rows.append(new_row)
    [row for row in rows if row.isnull().all(axis=None) or row.empty]
    # Конкатенируем все строки в один DataFrame
    final_df = pd.concat(rows, ignore_index=True)
    return final_df

expected_columns = [
    'Block Number', 'Trial Number', 'Total Trial Number', 'Targetword', 'Category', 
    'Index', 'Trial Result', 'Pressed Button', 'RT', 'Button Bad', 'Button Good', 'Button Vac'
]
def find_header_row(excel_file):
    data = pd.read_excel(excel_file, header=None)  # Читаем файл без заголовков
    for i, row in enumerate(data.values):
        # Проверяем, что все ожидаемые столбцы присутствуют в этой строке
        if all(col in row for col in expected_columns):
            return i
    return None

In [11]:
def preprocessing_participant_data(excel_file, participant, pivot_df):
    xls = pd.ExcelFile(excel_file)
    header_row = find_header_row(excel_file)

    if header_row is None:
        logging.error(f"Ошибка: не удалось найти строку с заголовками в файле участника {participant}")
        return None
    
    base_report_df = pd.read_excel(xls, sheet_name=0, skiprows=header_row)
    logging.info('Удаление незначимых строк...')
    base_report_df = base_report_df[~base_report_df['Block Number'].apply(lambda x: isinstance(x, str) and 'Check' in x)]
    logging.info(f'Итоговый предочищенный размер таблицы составил:\n\tСтолбцов: {base_report_df.shape[0]}, Строк: {base_report_df.shape[1]}')

    logging.info('Создание столбца: Block Number Modified...')
    base_report_df['Block Number Modified'] = base_report_df['Block Number'].apply(modify_block_number)
    logging.info('Создание столбца: IAT_EXP...')
    base_report_df['IAT_EXP'] = base_report_df['Block Number Modified'].apply(add_iat_exp)
    
    base_report_df = process_trials(base_report_df)
    base_report_df.rename(columns={'RT': 'IAT_RT'}, inplace=True)
    logging.info('Создание столбца: IAT_ACCURACY...')
    base_report_df['IAT_ACCURACY'] = base_report_df['Trial Result'].apply(add_iat_acc)

    logging.info('Считывание переменных IAT_results, IAT_results2, IAT_streght из pivot таблицы...')
    participant_data = pivot_df[pivot_df['ID'] == participant]
    if not participant_data.empty:
        base_report_df['IAT_results'] = participant_data['IAT_results'].values[0]
        base_report_df['IAT_streght'] = participant_data['IAT_streght'].values[0]
        base_report_df['IAT_results2'] = participant_data['IAT_results2'].values[0]

    logging.info('Создание столбцов Trial Valence, IAT_cb...')
    base_report_df['Trial Valence'] = base_report_df.apply(add_trial_valence, axis=1)
    base_report_df['IAT_cb'] = base_report_df.apply(determine_iat_cb, axis=1)

    fixation_sheets = [sheet for sheet in xls.sheet_names if sheet.startswith('FixationList')]
    saccade_sheets = [sheet for sheet in xls.sheet_names if sheet.startswith('SaccadeList')]
    logging.info(f'Size of Fixation List: {len(fixation_sheets)}')
    logging.info(f'Size of Saccade List: {len(saccade_sheets)}')
    logging.info(f'Size of Dataset: {len(base_report_df)}')

    
    final_data = Vac2Dataset(base_report_df, xls, fixation_sheets, saccade_sheets)
    agg_funcs = {
        'f_Duration': ['count', 'first', 'mean', 'sum'],
        's_Duration': ['count', 'mean', 'sum'],
        's_Amplitude': ['mean', 'sum'],
        's_Distance': ['mean', 'sum'],
        's_Peak Velocity': ['mean', 'sum']
    }
    # Агрегация по метрикам
    metrics = final_data.groupby('Total Trial Number').agg(agg_funcs)
    # Переименование столбцов для удобства
    metrics.columns = [
        'fixation_count', 'first_FD', 'mean_FD', 'total_FD',
        'saccades_count', 'mean_SD', 'total_SD',
        'mean_SA', 'total_SA',
        'mean_SDist', 'total_SDist',
        'mean_PV', 'total_PV'
    ]
    # Сброс индекса, чтобы сохранить trial как столбец
    metrics = metrics.reset_index()
    metrics = pd.merge(base_report_df, metrics, on='Total Trial Number', how='left')
    
    logging.info('Сортировка столбцов...')
    columns_order = ['Block Number', 'Trial Valence', 'IAT_results', 'IAT_results2', 'IAT_cb', 'IAT_streght', 'IAT_EXP',
                 'Trial Number', 'Total Trial Number', 'Block Number Modified', 'Targetword', 'Category', 'Index', 
                 'Trial Result', 'IAT_ACCURACY', 'Pressed Button', 'IAT_RT', 'Button Bad', 'Button Good', 'Button Vac', 
                 'f_Index', 'f_Center X', 'f_Center Y', 'f_Start Time', 'f_Duration', 'f_End Time',
                 's_Index', 's_Start Time', 's_Start X', 's_Start Y', 's_Duration', 's_Amplitude', 's_Distance', 's_Peak Velocity',
                 's_Average Velocity', 's_End Time', 's_End X','s_End Y',]
    
    if len(columns_order) != final_data.shape[1]:
        logging.warning(f'Will be dropped! {[j for j in final_data.columns if not (j in  columns_order)]}')
    final_data = final_data[columns_order]

    final_data.reset_index(drop=True, inplace=True)
    return final_data, metrics

In [12]:
# Путь к основной папке с участниками
participants_dir = 'EEG_CB_IAT'  # Замените на ваш путь
pivot_file  = 'CB_EEG_RSF.xlsx'

temp_dir = 'temp'
# Проверяем, существует ли папка, и если нет, создаем её
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
    logging.info(f"Папка {temp_dir} создана.")
else:
    logging.info(f"Папка {temp_dir} уже существует.")
dataset_metrcis_list = []
dataset_list = []
names_participant = []
# Проход по каждой папке с участником
for participant in os.listdir(participants_dir):
    participant_path = os.path.join(participants_dir, participant)

    # Проверяем, что это папка
    if os.path.isdir(participant_path):
        vac_2_path = os.path.join(participant_path, 'Vac_2')
        pivot_df = pd.read_excel(pivot_file)
        # Проверяем наличие папки Vac_2 внутри папки участника
        if not os.path.exists(vac_2_path) or not os.path.isdir(vac_2_path):
            logging.error(f"Ошибка: У участника {participant} отсутствует папка Vac_2")
        else:
            # Получаем список файлов в папке Vac_2
            files = os.listdir(vac_2_path)
            
            # Проверяем количество файлов в папке Vac_2
            if len(files) != 1:
                logging.error(f"Ошибка: У участника {participant} в папке Vac_2 неправильное количество файлов (найдено {len(files)})")

            elif pivot_df[pivot_df['ID'] == participant].empty:
                logging.error(f"Ошибка: Участник с ID {participant} не найден в {pivot_file}")

            #elif os.path.exists(f'temp/{participant}.xlsx'):
            #    logging.info(f'Уже существует {participant}')

            else:
                participant_data, metrics_data = preprocessing_participant_data(os.path.join(vac_2_path, files[0]), participant, pivot_df)
                if not (participant_data is None):
                    logging.info(f'Данные участника {participant} обработаны!')
                    dataset_list.append(participant_data)
                    dataset_metrcis_list.append(metrics_data)
                    names_participant.append(participant)

100%|██████████| 409/409 [00:02<00:00, 150.85it/s]
C:\Users\searg\AppData\Local\Temp\ipykernel_24236\297160089.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(rows, ignore_index=True)
100%|██████████| 397/397 [00:02<00:00, 158.87it/s]
C:\Users\searg\AppData\Local\Temp\ipykernel_24236\297160089.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(rows, ignore_index=True)
100%|██████████| 429/429 [00:06<00:00, 70.49it/s]
C:\Users\searg\AppData\Local

In [13]:
# Вывод списков
logging.info(f'Список участников: {names_participant}')

# Добавляем имя участника как новый столбец в каждый DataFrame и объединяем таблицы
for i, df in enumerate(dataset_list):
    df['Participant'] = os.path.splitext(names_participant[i])[0]
    # Перемещаем столбец 'Participant' на первое место
    cols = df.columns.tolist()
    cols = ['Participant'] + [col for col in cols if col != 'Participant']
    dataset_list[i] = df[cols]

    dataset_metrcis_list[i]['Participant'] = os.path.splitext(names_participant[i])[0]
    # Перемещаем столбец 'Participant' на первое место
    cols = dataset_metrcis_list[i].columns.tolist()
    cols = ['Participant'] + [col for col in cols if col != 'Participant']
    dataset_metrcis_list[i] = dataset_metrcis_list[i][cols]

# Объединяем все таблицы в одну
combined_df = pd.concat(dataset_list, ignore_index=True)
combined_metrics_df = pd.concat(dataset_metrcis_list, ignore_index=True)
combined_df.to_excel('IAT_RAW_EYE.xlsx', index=False)
combined_metrics_df.to_excel('IAT_EYE.xlsx', index=False)
logging.info('Датасет успешно создан!')

In [14]:
# Открываем файл и фильтруем строки уровня "ERROR"
error_lines = []

with open('log.txt', 'r') as log_file:
    for line in log_file:
        if 'ERROR' in line:  # Фильтруем по уровню ошибки
            error_lines.append(line)

# Выводим все строки с ошибками
for error in error_lines:
    print(error)
